In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
#use mistralai/Mistral-7B-Instruct-v0.3
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-Instruct-v0.3",
    max_seq_length = 2048,   # Context length
    load_in_4bit = True, #
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.6: Fast Mistral patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com

model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,           # Set rank of the LoRA matrices.
    lora_alpha = 16,  # Set scaling factor to 16
    lora_dropout = .05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,   # Rank stabilized LoRA
    loftq_config = None,  # LoftQ quantization
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.9.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("pubmed_qa", "pqa_artificial")

# Formatting promt
def format_prompt(example):

    # Set the prompt to be a knowledgeable biomedical assistant.
  prompt = f"""<|im_start|>system
  You are a knowledgeable biomedical assistant. Read the provided scientific context and answer the user’s question with a clear, detailed explanation in 2–3 paragraphs.<|im_end|>
  <|im_start|>user
  Question: {example['question']}<|im_end|>
  <|im_start|>assistant
  {example['long_answer']}<|im_end|>"""
  return {"text": prompt}

# Apply the new formatting function
formatted_dataset = dataset.map(
    format_prompt,
    remove_columns=list(dataset["train"].features),
)

README.md: 0.00B [00:00, ?B/s]

pqa_artificial/train-00000-of-00001.parq(…):   0%|          | 0.00/233M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

Map:   0%|          | 0/211269 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset["train"],
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 40,
        learning_rate = 2e-4,
        logging_steps = 2,
        optim="paged_adamw_32bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        report_to = "none",
    ),
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 211,269 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
2,0.000100
4,0.000000
6,0.000000
8,0.000200
10,0.000000
12,0.004500
14,0.000100
16,0.000100
18,0.002000
20,0.001100


In [ ]:
from transformers import TextStreamer


model = trainer.model

#Give chat template structure

system_prompt = "You are a helpful biomedical assistant. Your task is to answer the given question and provide scientific context and answer the user’s question with a clear, detailed explanation in 2–3 paragraphs"

user_question = "What are the symptoms of diabetes"


user_prompt = f"Question: {user_question}"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

# Apply the chat template
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False, # This prevents the <think>...</think> block
)

# Tokenize the prompt and prepare for streaming generation
# Move the tokenized inputs to the GPU where the model is.
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# The TextStreamer will print the output token-by-token to the console for a live, typewriter-like effect.
streamer = TextStreamer(tokenizer, skip_prompt=True)

# Generate the response
print("\n" + "="*50)
print("       FINE-TUNED MODEL RESPONSE (Direct Output)")
print("="*50 + "\n")

_ = model.generate(
    **model_inputs,
    streamer=streamer,
    max_new_tokens=10000, # Increased for detailed full answers
    temperature=0.6,
    top_p=0.9,
    do_sample=True, # Recommended for more natural-sounding text
)


       FINE-TUNED MODEL RESPONSE (Direct Output)

Group 2 innate lymphoid cells (ILC2s) are a subset of innate lymphoid cells that play a crucial role in the immune response to helminth infections and allergic diseases, particularly in the mucosal tissues of the respiratory and gastrointestinal tracts. ILC2s are activated by various cytokines, such as IL-25, IL-33, and thymic stromal lymphopoietin (TSLP), and they produce cytokines such as IL-5 and IL-13, which promote eosinophil recruitment, activation, and survival.

Chronic rhinosinusitis with nasal polyps (CRSwNP) is a chronic inflammatory disease of the nasal and paranasal sinuses characterized by the presence of nasal polyps and eosinophilia. Eosinophilia is an increase in the number of eosinophils, a type of white blood cell, in the blood or tissues. In the context of CRSwNP, eosinophilia is associated with increased levels of ILC2s and their cytokines, such as IL-5 and IL-13.

Several studies have investigated the role of ILC2

In [ ]:
#Save LoRA adapter and tokenizer for Gradio app
OUTPUT_DIR = "./biomed-lora"

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"LoRA adapter + tokenizer saved to {OUTPUT_DIR}")

LoRA adapter + tokenizer saved to ./biomed-lora


In [ ]:
import os
print(os.listdir("./biomed-lora"))

['tokenizer.model', 'README.md', 'adapter_model.safetensors', 'chat_template.jinja', 'tokenizer_config.json', 'tokenizer.json', 'adapter_config.json', 'special_tokens_map.json']


In [ ]:
# Zip the folder
!zip -r biomed-lora.zip biomed-lora

# Download the zip to your computer
from google.colab import files
files.download("biomed-lora.zip")

  adding: biomed-lora/ (stored 0%)
  adding: biomed-lora/tokenizer.model (deflated 61%)
  adding: biomed-lora/README.md (deflated 65%)
  adding: biomed-lora/adapter_model.safetensors (deflated 8%)
  adding: biomed-lora/chat_template.jinja (deflated 74%)
  adding: biomed-lora/tokenizer_config.json (deflated 96%)
  adding: biomed-lora/tokenizer.json (deflated 85%)
  adding: biomed-lora/adapter_config.json (deflated 57%)
  adding: biomed-lora/special_tokens_map.json (deflated 77%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>